#  Segmenting and Clustering Neighborhoods in Toronto (Part 2)

This notebook is part of Applied Data Science Capstone course @Coursera.

In part 1, Postal data was scraped, engineered, having Latitude and Longitude added, and saved to CSV. Here we load the CSV back in to continue working on it.

In [12]:
import pandas as pd
df = pd.read_csv('Canada_Postal.csv', index_col=0)
print(df.head())
print(df.shape)

  PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  
(103, 5)


## Exploring data set

We are going to visualize the neighborhood with map. 

In [13]:
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

First we find the average of latitude/longtitude to use as center of our map.

In [22]:
latitude = df2.Latitude.mean()
longitude = df2.Longitude.mean()
print('The coordinates we are using are {}, {}.'.format(latitude, longitude))

The coordinates we are using are 43.704607733980595, -79.3971529116505.


A few test runs revealed zoom_start=11 to be the ideal number

In [23]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
map

Narrow down the scope to only Borough containing the word Toronto.

In [34]:
toronto = df2[df2['Borough'].str.contains('Toronto')]
toronto.reset_index(drop=True, inplace=True)
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [35]:
latitude = toronto.Latitude.mean()
longitude = toronto.Longitude.mean()
map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
map

## Exploring neighborhood

I picked Cabbagetown to explore because I like its name.


In [69]:
f = open('fsquare.txt', 'r')
ID, SECRET = f.readlines()
ID = ID.rstrip()
SECRET = SECRET.rstrip()
version = '20201101'


In [63]:
cablat = toronto[toronto['Neighborhood'].str.contains('Cabbagetown')].Latitude.values[0]
cablong = toronto[toronto['Neighborhood'].str.contains('Cabbagetown')].Longitude.values[0]

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={ID}&client_secret={SECRET}&ll={cablat},{cablong}&radius=500&limit=100&v={version}'

results = requests.get(url).json()

Reusing function from of the labs in our class.

In [64]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning json and structure it into a _pandas_ dataframe.

In [67]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cranberries,Diner,43.667843,-79.369407
1,Kingyo Toronto,Japanese Restaurant,43.665895,-79.368415
2,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
3,Murgatroid,Restaurant,43.667381,-79.369311
4,F'Amelia,Italian Restaurant,43.667536,-79.368613


Let's see how many venues we got

In [68]:
nearby_venues.shape[0]

48

## Explore more neighborhoods

Function again reused from class lab

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={ID}&client_secret={SECRET}&ll={cablat},{cablong}&radius=500&limit=100&v={version}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
# utilizing the above function
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Let's check our new data frame

In [73]:
print(toronto_venues.shape)
toronto_venues.head()

(1872, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Cranberries,43.667843,-79.369407,Diner
1,"Regent Park, Harbourfront",43.65426,-79.360636,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
2,"Regent Park, Harbourfront",43.65426,-79.360636,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,Murgatroid,43.667381,-79.369311,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,F'Amelia,43.667536,-79.368613,Italian Restaurant


Venues returned per neighborhood

In [74]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,48,48,48,48,48,48
"Brockton, Parkdale Village, Exhibition Place",48,48,48,48,48,48
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",48,48,48,48,48,48
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",48,48,48,48,48,48
Central Bay Street,48,48,48,48,48,48
Christie,48,48,48,48,48,48
Church and Wellesley,48,48,48,48,48,48
"Commerce Court, Victoria Hotel",48,48,48,48,48,48
Davisville,48,48,48,48,48,48


#### Let's find out how many unique categories can be curated from all the returned venues

In [75]:
unique = len(toronto_venues['Venue Category'].unique())
print(f'There are {unique} uniques categories.')

There are 33 uniques categories.


## Neighborhood Analyzing

In [76]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Bakery,Bank,Beer Store,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Deli / Bodega,...,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Snack Place,Taiwanese Restaurant,Thai Restaurant
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# checking size
toronto_onehot.shape

(1872, 34)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [82]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()


,Neighborhood,Bakery,Bank,Beer Store,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Deli / Bodega,...,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Snack Place,Taiwanese Restaurant,Thai Restaurant
0,Berczy Park,0.041667,0.020833,0.020833,0.020833,0.0625,0.020833,0.041667,0.083333,0.020833,...,0.020833,0.0625,0.020833,0.020833,0.041667,0.0625,0.020833,0.020833,0.020833,0.020833
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.020833,0.020833,0.020833,0.0625,0.020833,0.041667,0.083333,0.020833,...,0.020833,0.0625,0.020833,0.020833,0.041667,0.0625,0.020833,0.020833,0.020833,0.020833
2,"Business reply mail Processing Centre, South C...",0.041667,0.020833,0.020833,0.020833,0.0625,0.020833,0.041667,0.083333,0.020833,...,0.020833,0.0625,0.020833,0.020833,0.041667,0.0625,0.020833,0.020833,0.020833,0.020833
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.041667,0.020833,0.020833,0.020833,0.0625,0.020833,0.041667,0.083333,0.020833,...,0.020833,0.0625,0.020833,0.020833,0.041667,0.0625,0.020833,0.020833,0.020833,0.020833
4,Central Bay Street,0.041667,0.020833,0.020833,0.020833,0.0625,0.020833,0.041667,0.083333,0.020833,...,0.020833,0.0625,0.020833,0.020833,0.041667,0.0625,0.020833,0.020833,0.020833,0.020833


In [83]:
# check size again
toronto_grouped.shape

(39, 34)

### Top 5 most common venue of each neighborhood

In [84]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.06
2         Café  0.06
3  Pizza Place  0.06
4       Bakery  0.04


----Christie----
         venue  freq
0  Coffee Shop

### turn into dataframe

In [87]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
4,Central Bay Street,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub


## Clustering neighborhoods

In [88]:
from sklearn.cluster import KMeans

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Creating new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [91]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pizza Place,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Market,Park,Bakery,Pub


Finally, visualization

In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters